# Thư viện cần thiết

In [19]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold, RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Load dữ liệu

In [20]:
X_train = pd.read_csv('../exps/feature_Labels/ex1_train.csv')
X_test = pd.read_csv('../exps/feature_Labels/ex1_test.csv')
y = pd.read_csv('../exps/feature_Labels/y_train.csv')

# Tối ưu hóa, Huấn luyện và test

In [21]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

## Model mặc định:

In [22]:
svr = SVR()
rfr = RandomForestRegressor()
gbm = GradientBoostingRegressor()

In [23]:
models = [('SVR', svr), ('RandomForest', rfr), ('GradientBoosting', gbm)]
results = {}
for name, model in models:
    # cross_val_score trả về điểm âm vì greater_is_better=False, 
    # nên ta lấy giá trị âm của nó để có RMSE dương     
    scores = -cross_val_score(
        estimator=model,
        X=X_train,
        y=y,
        scoring=rmse_scorer,
        cv=kf,
        n_jobs=-1
    )
    results[name] = scores.mean()
    print(f"| {name:<16} | RMSE TB: {scores.mean():.4f} |")


| SVR              | RMSE TB: 0.1153 |
| RandomForest     | RMSE TB: 0.1278 |
| GradientBoosting | RMSE TB: 0.1150 |


## Tối ưu hóa mô hình:

In [24]:
# === TỐI ƯU HÓA MÔ HÌNH VỚI RANDOMIZEDSEARCHCV ===

def random_search(model, grid, n_iter=100):
    search = RandomizedSearchCV(
        estimator=model, 
        param_distributions=grid, 
        cv=kf, 
        n_iter=n_iter, 
        n_jobs=-1, 
        random_state=0, 
        verbose=1,
        scoring=rmse_scorer
    )
    return search.fit(X_train, y.values.ravel())

# Define hyperparameter grids
svr_hpg = {
    "C": np.arange(1, 100), 
    "gamma": np.linspace(0.00001, 0.001, 50), 
    "epsilon": np.linspace(0.01, 0.1, 50)
}

gbm_hpg = {
    "max_features": np.linspace(0.2, 0.7, 6), 
    "learning_rate": np.logspace(-3, -1, 100),
    "n_estimators": [100, 200, 500, 1000],
    "max_depth": [3, 5, 7, 9]
}

rfr_hpg = {
    "n_estimators": [100, 300, 500, 1000], 
    "max_features": np.linspace(0.1, 0.5, 5),
    "max_depth": [10, 20, 30, None], 
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

In [25]:
# Tối ưu RandomForest
print("=== TỐI ƯU RANDOM FOREST ===")
rfr_search = random_search(RandomForestRegressor(), rfr_hpg, n_iter=50)
print(f"Best params: {rfr_search.best_params_}")
print(f"Best score: {-rfr_search.best_score_:.4f}")

# Tối ưu SVR
print("\n=== TỐI ƯU SVR ===")
svr_search = random_search(SVR(), svr_hpg, n_iter=50)
print(f"Best params: {svr_search.best_params_}")
print(f"Best score: {-svr_search.best_score_:.4f}")

# Tối ưu Gradient Boosting
print("\n=== TỐI ƯU GRADIENT BOOSTING ===")
gbm_search = random_search(GradientBoostingRegressor(), gbm_hpg, n_iter=50)
print(f"Best params: {gbm_search.best_params_}")
print(f"Best score: {-gbm_search.best_score_:.4f}")


=== TỐI ƯU RANDOM FOREST ===
Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best params: {'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': np.float64(0.2), 'max_depth': None}
Best score: 0.1216

=== TỐI ƯU SVR ===
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best params: {'gamma': np.float64(0.0001716326530612245), 'epsilon': np.float64(0.032040816326530615), 'C': np.int64(29)}
Best score: 0.1065

=== TỐI ƯU GRADIENT BOOSTING ===
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best params: {'n_estimators': 1000, 'max_features': np.float64(0.5), 'max_depth': 3, 'learning_rate': np.float64(0.03944206059437656)}
Best score: 0.1068


In [26]:
rfr_optimized = rfr_search.best_estimator_ 
svr_optimized = svr_search.best_estimator_
gbm_optimized = gbm_search.best_estimator_

## Học tập kết hợp (Ensemble Learning)

**Sử dụng phương pháp stacking(xếp chồng):**

Giảm thiểu Sai số (Tăng Độ chính xác)

Tận dụng Điểm mạnh: Mỗi mô hình hồi quy (như SVR, Random Forest, Gradient Boosting) đều có cách tiếp cận dữ liệu và lỗi khác nhau:

SVR giỏi trong việc tìm ra biên (boundary) và xử lý dữ liệu ở không gian nhiều chiều.

Random Forest giỏi trong việc xử lý dữ liệu phi tuyến tính và chống nhiễu (noise) nhờ vào tính ngẫu nhiên của nó.

Gradient Boosting giỏi trong việc tập trung vào các điểm dữ liệu khó (những nơi mà các mô hình trước đó dự đoán sai).

In [27]:
base_estimators = [
    ('svr', svr_optimized), 
    ('rfr', rfr_optimized),
    ('gbm', gbm_optimized)
]

In [28]:
# Mô hình này sẽ học cách kết hợp dự đoán từ 3 mô hình trên.
# RidgeCV hoặc LinearRegression thường là lựa chọn tốt.
meta_model = RidgeCV()

In [29]:
stacked_model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=meta_model,
    cv=kf,
    n_jobs=-1 
)

## Huấn luyện mô hình

In [30]:
stacked_model.fit(X_train,y)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_stacking.py:970: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingRegressor(cv=KFold(n_splits=5, random_state=0, shuffle=True),
                  estimators=[('svr',
                               SVR(C=np.int64(29),
                                   epsilon=np.float64(0.032040816326530615),
                                   gamma=np.float64(0.0001716326530612245))),
                              ('rfr',
                               RandomForestRegressor(max_features=np.float64(0.2),
                                                     min_samples_split=5,
                                                     n_estimators=1000)),
                              ('gbm',
                               GradientBoostingRegressor(learning_rate=np.float64(0.03944206059437656),
                                                         max_features=np.float64(0.5),
                                                         n_estimators=1000))],
                  final_estimator=RidgeCV(), n_jobs=-1)

In [31]:
prediction = stacked_model.predict(X_test)
#apply rescale lại cho output đã đc xử lí
prediction = np.exp(prediction)

# Submission

In [32]:
test = pd.read_csv('../data/test.csv')

In [35]:
my_prediction = pd.DataFrame({"Id": test['Id'], "SalePrice": prediction})
my_prediction.to_csv('../tmp_sub/exp1.csv',index=False)

# Testing từng model

## Random Forest Regression

In [36]:
rfr_optimized.fit(X_train,y)
prediction = rfr_optimized.predict(X_test)
rfr_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
rfr_pred.to_csv('../submission/rfr_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


## Support Vector Regression

In [37]:
svr_optimized.fit(X_train,y)
prediction = svr_optimized.predict(X_test)
svr_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
svr_pred.to_csv('../submission/svr_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Gradient Boosting

In [38]:
gbm_optimized.fit(X_train,y)
prediction = gbm_optimized.predict(X_test)
gbm_pred = pd.DataFrame({"Id": test['Id'], "SalePrice": np.exp(prediction)})
gbm_pred.to_csv('../submission/gbm_optimized.csv',index=False)

c:\Users\trand\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
